### Install dependencies

In [1]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U

### Load CSV data from S3

In [2]:
!pwd

/home/ec2-user/SageMaker/Enterprise-RAG/notebooks


In [3]:
import boto3
import pandas as pd

s3 = boto3.client('s3')
bucket_name = 'recipes-rag'

In [4]:
# file_key = 'food_recipes.csv'

# # Download the file from s3 locally
# s3.download_file(bucket_name, file_key, '../data/food_recipes.csv')

# # Load the CSV into a DataFrame
# df = pd.read_csv('../data/food_recipes.csv')

# df.head()

In [5]:
file_key = 'recipes_w_cleaning_time_combined_features.parquet'
s3.download_file(bucket_name, file_key, f'../data/{file_key}')
df = pd.read_parquet(f'../data/{file_key}')

df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,CookTime_Minutes,PrepTime_Minutes,TotalTime_Minutes,Combined_Features_Clean
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo...",1440,45,1485,Low-Fat Berry Blue Frozen Dessert Frozen Desse...
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...,25,240,265,Biryani Chicken Breast Make share Biryani reci...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu...",5,30,35,Best Lemonade Beverages one first Good House K...
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces...",20,1440,1460,Carina's Tofu-Vegetable Kebabs Soy/Tofu dish b...
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,...",30,20,50,Cabbage Soup Vegetable Make share Cabbage Soup...


In [6]:
df.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions',
       'CookTime_Minutes', 'PrepTime_Minutes', 'TotalTime_Minutes',
       'Combined_Features_Clean'],
      dtype='object')

In [7]:
str(df.iloc[0]['Combined_Features_Clean'])

'Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy'

### Load data into chunked documents

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
text_splitter_recursive = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)

In [11]:
# Using old raw dataset

# def create_documents(df):
#     # AWS Kendra requires non empty string values for each field
#     # Make explicit to the LLM that the field is not available
#     df.fillna('Not Available', inplace=True)
    
#     documents = []
#     for index, row in df.iterrows():
#         metadata = {
#             'recipe_title': str(row['recipe_title']) if row['recipe_title'] else 'No Title Available',
#             'url': str(row['url']) if row['url'] else 'https://example.com',
#             # 'record_health': str(row['record_health']) if row['record_health'] else 'Unknown',
#             'vote_count': str(row['vote_count']) if row['vote_count'] else 'No Votes Available',
#             'rating': str(row['rating']) if row['rating'] else 'No Rating Available',
#             'cuisine': str(row['cuisine']) if row['cuisine'] else 'No Cuisine Available',
#             'course': str(row['course']) if row['course'] else 'No Course Available',
#             'diet': str(row['diet']) if row['diet'] else 'No Diet Information Available',
#             'prep_time': str(row['prep_time']) if row['prep_time'] else 'No Prep Time Available',
#             'cook_time': str(row['cook_time']) if row['cook_time'] else 'No Cook Time Available',
#             'author': str(row['author']) if row['author'] else 'No Author Available',
#             'category': str(row['category']) if row['category'] else 'No Category Available'
#         }

#         # Combine all text fields for the document content
#         text = f"{row['description']} {row['ingredients']} {row['instructions']} {row['tags']}"
#         doc = Document(page_content=text, metadata=metadata)
#         documents.append(doc)
    
#     return documents

In [12]:
# Using data after EDA with more complex fields

# def create_documents(df):
#     # Ensure non-empty string values for each field
#     df.fillna('Not Available', inplace=True)
    
#     documents = []
#     for index, row in df.iterrows():
#         metadata = {
#             'recipe_id': str(row['RecipeId']) if row['RecipeId'] else 'No ID Available',
#             'name': str(row['Name']) if row['Name'] else 'No Name Available',
#             'author_id': str(row['AuthorId']) if row['AuthorId'] else 'No Author ID Available',
#             'author_name': str(row['AuthorName']) if row['AuthorName'] else 'No Author Name Available',
#             'cook_time': str(row['CookTime']) if row['CookTime'] else 'No Cook Time Available',
#             'prep_time': str(row['PrepTime']) if row['PrepTime'] else 'No Prep Time Available',
#             'total_time': str(row['TotalTime']) if row['TotalTime'] else 'No Total Time Available',
#             'date_published': str(row['DatePublished']) if row['DatePublished'] else 'No Date Available',
#             'recipe_category': str(row['RecipeCategory']) if row['RecipeCategory'] else 'No Category Available',
#             'keywords': str(row['Keywords']) if row['Keywords'] else 'No Keywords Available',
#             'aggregated_rating': str(row['AggregatedRating']) if row['AggregatedRating'] else 'No Rating Available',
#             'review_count': str(row['ReviewCount']) if row['ReviewCount'] else 'No Reviews Available',
#             'calories': str(row['Calories']) if row['Calories'] else 'No Calories Information Available',
#             'fat_content': str(row['FatContent']) if row['FatContent'] else 'No Fat Content Available',
#             'saturated_fat_content': str(row['SaturatedFatContent']) if row['SaturatedFatContent'] else 'No Saturated Fat Content Available',
#             'cholesterol_content': str(row['CholesterolContent']) if row['CholesterolContent'] else 'No Cholesterol Content Available',
#             'sodium_content': str(row['SodiumContent']) if row['SodiumContent'] else 'No Sodium Content Available',
#             'carbohydrate_content': str(row['CarbohydrateContent']) if row['CarbohydrateContent'] else 'No Carbohydrate Content Available',
#             'fiber_content': str(row['FiberContent']) if row['FiberContent'] else 'No Fiber Content Available',
#             'sugar_content': str(row['SugarContent']) if row['SugarContent'] else 'No Sugar Content Available',
#             'protein_content': str(row['ProteinContent']) if row['ProteinContent'] else 'No Protein Content Available',
#             'recipe_servings': str(row['RecipeServings']) if row['RecipeServings'] else 'No Servings Information Available',
#             'recipe_yield': str(row['RecipeYield']) if row['RecipeYield'] else 'No Yield Information Available'
#         }

#         # Combine relevant text fields for the document content
#         text = f"""
#         Name: {row['Name']}
#         Category: {row['RecipeCategory']}
#         Description: {row['Description']}
#         Keywords: {row['Keywords']}
#         Ingredients: {row['RecipeIngredientParts']}
#         Instructions: {row['RecipeInstructions']}
#         """

#         doc = Document(page_content=text.strip(), metadata=metadata)
#         documents.append(doc)
    
#     return documents


In [13]:
def create_documents(df):
    documents = []
    for index, row in df.iterrows():
        metadata = {
            'recipe_id': str(row['RecipeId']) if not pd.isna(row['RecipeId']) else 'No ID Available',
            'name': str(row['Name']) if not pd.isna(row['Name']) else 'No Name Available',
            'cook_time': str(row['CookTime']) if not pd.isna(row['CookTime']) else 'No Cook Time Available',
            'prep_time': str(row['PrepTime']) if not pd.isna(row['PrepTime']) else 'No Prep Time Available',
            'total_time': str(row['TotalTime']) if not pd.isna(row['TotalTime']) else 'No Total Time Available',
            'recipe_category': str(row['RecipeCategory']) if not pd.isna(row['RecipeCategory']) else 'No Category Available',
            'keywords': str(row['Keywords']) if not pd.isna(row['Keywords']).all() else 'No Keywords Available',
            'aggregated_rating': str(row['AggregatedRating']) if not pd.isna(row['AggregatedRating']) else 'No Rating Available',
            'review_count': str(row['ReviewCount']) if not pd.isna(row['ReviewCount']) else 'No Reviews Available',
            'calories': str(row['Calories']) if not pd.isna(row['Calories']) else 'No Calories Information Available',
            'fat_content': str(row['FatContent']) if not pd.isna(row['FatContent']) else 'No Fat Content Available',
            'saturated_fat_content': str(row['SaturatedFatContent']) if not pd.isna(row['SaturatedFatContent']) else 'No Saturated Fat Content Available',
            'cholesterol_content': str(row['CholesterolContent']) if not pd.isna(row['CholesterolContent']) else 'No Cholesterol Content Available',
            'sodium_content': str(row['SodiumContent']) if not pd.isna(row['SodiumContent']) else 'No Sodium Content Available',
            'carbohydrate_content': str(row['CarbohydrateContent']) if not pd.isna(row['CarbohydrateContent']) else 'No Carbohydrate Content Available',
            'sugar_content': str(row['SugarContent']) if not pd.isna(row['SugarContent']) else 'No Sugar Content Available',
            'protein_content': str(row['ProteinContent']) if not pd.isna(row['ProteinContent']) else 'No Protein Content Available',
            'recipe_servings': str(row['RecipeServings']) if not pd.isna(row['RecipeServings']) else 'No Servings Information Available',
            'recipe_yield': str(row['RecipeYield']) if not pd.isna(row['RecipeYield']) else 'No Yield Information Available'
        }

        # Use Combined_Features_Clean for the document content
        text = str(row['Combined_Features_Clean'])
        doc = Document(page_content=text, metadata=metadata)
        documents.append(doc)
        
    return documents


In [14]:
documents = create_documents(df)

In [15]:
documents[0]

Document(metadata={'recipe_id': '38.0', 'name': 'Low-Fat Berry Blue Frozen Dessert', 'cook_time': 'PT24H', 'prep_time': 'PT45M', 'total_time': 'PT24H45M', 'recipe_category': 'Frozen Desserts', 'keywords': "['Dessert' 'Low Protein' 'Low Cholesterol' 'Healthy' 'Free Of...' 'Summer'\n 'Weeknight' 'Freezer' 'Easy']", 'aggregated_rating': '4.5', 'review_count': '4.0', 'calories': '170.9', 'fat_content': '2.5', 'saturated_fat_content': '1.3', 'cholesterol_content': '8.0', 'sodium_content': '29.8', 'carbohydrate_content': '37.1', 'sugar_content': '30.2', 'protein_content': '3.2', 'recipe_servings': '4.0', 'recipe_yield': 'No Yield Information Available'}, page_content='Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy')

In [16]:
len(documents)

522517

In [17]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [18]:
split_documents = split_documents_with_metadata(documents, text_splitter_recursive)

In [19]:
split_documents[0]

Document(metadata={'recipe_id': '38.0', 'name': 'Low-Fat Berry Blue Frozen Dessert', 'cook_time': 'PT24H', 'prep_time': 'PT45M', 'total_time': 'PT24H45M', 'recipe_category': 'Frozen Desserts', 'keywords': "['Dessert' 'Low Protein' 'Low Cholesterol' 'Healthy' 'Free Of...' 'Summer'\n 'Weeknight' 'Freezer' 'Easy']", 'aggregated_rating': '4.5', 'review_count': '4.0', 'calories': '170.9', 'fat_content': '2.5', 'saturated_fat_content': '1.3', 'cholesterol_content': '8.0', 'sodium_content': '29.8', 'carbohydrate_content': '37.1', 'sugar_content': '30.2', 'protein_content': '3.2', 'recipe_servings': '4.0', 'recipe_yield': 'No Yield Information Available', 'chunk_id': 0}, page_content='Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy')

In [20]:
# from transformers import AutoTokenizer

# def count_tokens(text, model_name="distilbert-base-uncased"):
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     tokens = tokenizer.encode(text, add_special_tokens=False)
#     num_tokens = len(tokens)
    
#     return num_tokens

In [21]:
# We have > 500,000 recipes, this takes a long time to run
from langchain_community.vectorstores import Qdrant

qdrant_store = Qdrant.from_documents(documents,
    embedding_model,
    location=":memory:",
)

In [22]:
qdrant_retriever = qdrant_store.as_retriever()

In [23]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

### Agent function definitions

In [24]:
recipe_db_query_tool = {
  "name": "query_food_recipe_vector_db",
  "description": """
      Queries the vector database containing food recipes to retrieve the most relevant documents. 
      This function allows the model to generate and execute multiple queries as necessary to gather comprehensive context, 
      such as ingredients, preparation steps, and metadata like cuisine and diet type, ensuring accurate and thorough responses to user queries.
      """,
  "input_schema": {
    "type": "object",
    "properties": {
      "queries": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "A query generated by the model to run against the vector database to fetch recipe documents."
        },
        "description": "A list of queries generated by the model to run against the vector database to fetch recipe documents."
      }
    },
    "required": ["queries"]
  }
}


### Init bedrock model, define util to stateless messaging, no fn calling

In [25]:
import json

In [26]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [27]:
# We will need to tune these prompts
# query_bedrock_llm() definition NEEDS TO BE RERUN
# each time when changes are made to this prompt

baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

Before answering, always make at least one call to query_food_recipe_vector_db
to retrieve the relevant context of recipes and ingredients to generate an informed
and high-quality response to the user prompt but NEVER exceed a MAXIMUM of 
3 calls to the query_food_recipe_vector_db function.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [28]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

### Pipe langchain together

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [30]:
baseline_user_prompt = """
### Here is a user prompt:
{query}
"""

In [31]:
def process_prompt(query_args):
    prompt_with_query = baseline_user_prompt.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

In [32]:
qdrant_rag_chain = (
    RunnableMap(
        # {"context": qdrant_retriever | format_docs,
         {"query": RunnablePassthrough()}
    )
    | process_prompt
    | query_bedrock_llm
    # | parse_event_stream
)

### Model generates dynamic context queries to vector db

In [33]:
test_query_1 = "I enjoy asian fusion food and I am a vegetarian. Give me one recipe with ingredients and instructions"

In [34]:
qdrant_rag_chain.invoke(test_query_1)

{'id': 'msg_bdrk_01V19vnku64nmMTKso6TjSAb',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': 'Okay, let me query the recipe database to find a relevant Asian fusion vegetarian recipe for you.'},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01VMdyVdYVKjJyCzLn3TWuft',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['asian fusion vegetarian recipes']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 472, 'output_tokens': 87}}

In [35]:
test_query_2 = """
I have a peanut allergy but I like thai food. 
I also don't enjoy spicy food much, and want a meal with low carbs. 
Give a recipe with ingredients and instructions
"""

In [36]:
qdrant_rag_chain.invoke(test_query_2)

{'id': 'msg_bdrk_0134txhPyGX1jBNWFRd59G7g',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': 'Okay, let me query the food recipe database to find a suitable Thai recipe that meets your requirements.'},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01GNxvPs95AkwzTtpxfAancs',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 496, 'output_tokens': 100}}

In [37]:
test_query_3 = """
Suggest a low-carb breakfast recipe that includes eggs and spinach, 
can be prepared in under 20 minutes, 
and is suitable for a keto diet.
"""

In [38]:
qdrant_rag_chain.invoke(test_query_3)

{'id': 'msg_bdrk_01EU9jHspsZCwFjDe5e4ZZ7S',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01SAet5TYX5VafG9DnyHH6RV',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['low-carb breakfast recipe with eggs and spinach',
     'keto breakfast recipe with eggs and spinach']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 491, 'output_tokens': 82}}

In [39]:
test_query_4 = """
Suggest a healthy dinner recipe for two people that includes fish, 
is under 500 calories per serving, 
and can be made in less than 40 minutes.
"""

In [40]:
qdrant_rag_chain.invoke(test_query_4)

{'id': 'msg_bdrk_01HLT8LcdvjTgowpw3fjHMQG',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': 'Okay, let me query the recipe database to find some healthy fish dinner options that meet those criteria.'},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01GQzCYxdEPHGkj6FSgTmKZU',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['healthy fish dinner recipe under 500 calories quick']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 490, 'output_tokens': 92}}

### Implement continuous dialogue and function calling

In [41]:
def generate_message(prompt):
    if type(prompt) != str:
        raise ValueError(f'Tried to call message generate_message with non-string input: {prompt}')
        
    return {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt
            }
        ]
    }

In [42]:
def generate_tool_message(fn_results):
    
    return {
        "role": "user",
        "content": fn_results
    }

In [43]:
import json

# Adds the current prompt as a new message to the chat history
# and calls bedrock with the entire chat history
# Returns the response body, llm's message, and new chat history

'''
Example response body structure:
{
   "id":"msg_bdrk_01C5GGkafK7aL3P5i3rsMr1p",
   "type":"message",
   "role":"assistant",
   "model":"claude-3-sonnet-20240229",
   "content":[
      {
         "type":"tool_use",
         "id":"toolu_bdrk_01CQiYa8BMJfpJC68DuRdwQn",
         "name":"query_food_recipe_vector_db",
         "input":{
            "queries":[
               "healthy fish dinner recipe under 500 calories",
               "fish dinner recipe for two under 40 minutes"
            ]
         }
      }
   ],
   "stop_reason":"tool_use",
   "stop_sequence":"None",
   "usage":{
      "input_tokens":559,
      "output_tokens":55
   }
}
'''
def message_handler(existing_chat_history, prompt, is_tool_message=False):
    # Fn results is an array of tool response objects
    # message structure needs to reflect that
    if is_tool_message:
        user_message = generate_tool_message(prompt)
    else:
        user_message = generate_message(prompt)
    existing_chat_history.append(user_message)

    # Parse the response content
    response_body = query_bedrock_llm(existing_chat_history)
    llm_message = {
        'role': response_body['role'],
        'content': response_body['content']
    }

    # Add the response message to the chat history
    existing_chat_history.append(llm_message)
    
    return [response_body, llm_message, existing_chat_history]

In [44]:
# Executes a list of queries and returns a list of document results
def handle_vector_db_queries(queries, retriever=qdrant_retriever): 
    context_docs = []
    for query in queries:
        query_results = retriever.invoke(query)
        context_docs.extend(query_results)

    return context_docs

In [45]:
# Takes as an argument to LLM message content, returns a list of the fn result objects
def handle_function_calls(tool_call_message_content):
    tool_results = []
    
    for tool_call in tool_call_message_content:
        # Only process messages from the LLM that are function calls
        if tool_call['type'] != 'tool_use':
            continue
        fn_id = tool_call['id']
        fn_name = tool_call['name']
        fn_args = tool_call['input']
        fn_result = {
            "type": "tool_result",
            "tool_use_id": fn_id,
        }   

        if fn_name == 'query_food_recipe_vector_db':
            if 'queries' not in fn_args:
                print(f"ERROR: Tried to call {fn_name} with invalid args {fn_args}, skipping..")
                fn_result['content'] = ""
                fn_result['is_error'] = True
                tool_results.append(fn_result)
                continue
                
            print(f"Model called {fn_name} with args {fn_args}")
            context_docs = handle_vector_db_queries(fn_args['queries'])
            context_str = format_docs(context_docs)
            fn_result['content'] = context_str
            tool_results.append(fn_result)
            
        # TODO: handle web search invocation here
        
        else:
            print(f"ERROR: Attempted call to unknown function {fn_name}")
            fn_result['content'] = ""
            fn_result['is_error'] = True
            tool_results.append(fn_result)

    return tool_results

### Run dynamic query function calling with test queries

In [46]:
'''
Example payload structure of response_body:

{'id': 'msg_bdrk_01REesjegNiLteurBoxW7pSt',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01191W2FuAFTRoDqKKeJSmmn',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 573, 'output_tokens': 52}}


Example payload structure of llm_message['content']:

[{'type': 'tool_use',
   'id': 'toolu_bdrk_01191W2FuAFTRoDqKKeJSmmn',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}]
'''

# This function is the entry point to invoke the LLM with support for function calling
# parsing output, calling requested functions, sending output is handled here
def run_chat_loop(prompt):
    print(f"[User]: {prompt}")
    response_body, llm_message, chat_history = message_handler(existing_chat_history=[], prompt=prompt)
    
    
    # The model wants to call tools, call them, provide response, repeat until content is generated
    while response_body['stop_reason'] == 'tool_use':
        fn_results = handle_function_calls(tool_call_message_content=llm_message['content'])

        # Send function results back to LLM as a new message with the existing chat history
        response_body, llm_message, chat_history = message_handler(
            existing_chat_history=chat_history, 
            prompt=fn_results,
            is_tool_message=True
        )

    # The model is done calling tools
    print(f"\n[Model]: {llm_message['content'][0]['text']}")


In [82]:
test_query_response_4 = run_chat_loop(test_query_4)

[User]: 
Suggest a healthy dinner recipe for two people that includes fish, 
is under 500 calories per serving, 
and can be made in less than 40 minutes.

Model called query_food_recipe_vector_db with args {'queries': ['healthy dinner recipe fish under 500 calories 40 minutes']}

[Model]: Based on the recipe metadata retrieved, I recommend the following healthy dinner recipe for two people that includes fish, is under 500 calories per serving, and can be made in less than 40 minutes:

Recipe: Tasty Fish
Calories per serving: 195.2
Total time: 50 minutes (40 minutes cook time)
Servings: 3

Ingredients:
- 2 fillets of white fish (such as tilapia or cod)
- 1 tbsp olive oil
- 1 tsp lemon juice
- 1 tsp dried oregano
- Salt and pepper to taste
- Steamed vegetables of your choice

Instructions:
1. Preheat oven to 400°F.
2. Pat the fish fillets dry and place them on a baking sheet lined with parchment paper.
3. Drizzle the fish with olive oil and lemon juice, then sprinkle with oregano, salt, 

In [83]:
test_query_response_3 = run_chat_loop(test_query_3)

[User]: 
Suggest a low-carb breakfast recipe that includes eggs and spinach, 
can be prepared in under 20 minutes, 
and is suitable for a keto diet.

Model called query_food_recipe_vector_db with args {'queries': ['low carb breakfast recipe eggs spinach keto']}

[Model]: Based on the search results, here is a recommended low-carb breakfast recipe with eggs and spinach that is keto-friendly and can be prepared in under 20 minutes:

Keto Egg Muffins

Ingredients:
- 12 eggs
- 1 cup fresh spinach, chopped
- 1/4 cup shredded cheddar cheese
- Salt and pepper to taste

Instructions:
1. Preheat oven to 350°F. Grease a 12-cup muffin tin.
2. In a large bowl, whisk the eggs. Stir in the chopped spinach and shredded cheese. Season with salt and pepper.
3. Divide the egg mixture evenly among the prepared muffin cups.
4. Bake for 15-18 minutes, until the eggs are set.
5. Allow the muffins to cool for 5 minutes before removing from the tin.

This recipe is perfect for a quick and easy low-carb breakf

In [84]:
test_query_response_2 = run_chat_loop(test_query_2)

[User]: 
I have a peanut allergy but I like thai food. 
I also don't enjoy spicy food much, and want a meal with low carbs. 
Give a recipe with ingredients and instructions

Model called query_food_recipe_vector_db with args {'queries': ['thai food recipe peanut allergy low carb not spicy']}

[Model]: Based on the search results, it looks like a good option for you would be a Thai Peanut Sauce recipe that is low in carbs and not too spicy. Here is a recipe that fits your needs:

Thai Peanut Sauce

Ingredients:
- 1/2 cup unsweetened coconut milk
- 2 tablespoons natural peanut butter (make sure it's peanut allergy-friendly)
- 1 tablespoon low-sodium soy sauce
- 1 tablespoon lime juice
- 1 teaspoon grated ginger
- 1/4 teaspoon garlic powder
- 1/4 teaspoon red pepper flakes (optional, omit if you don't want any spice)

Instructions:
1. In a small bowl, whisk together all the ingredients until well combined.
2. Taste and adjust seasoning as needed, adding more lime juice for acidity or soy 

In [85]:
test_query_response_1 = run_chat_loop(test_query_1)

[User]: I enjoy asian fusion food and I am a vegetarian. Give me one recipe with ingredients and instructions
Model called query_food_recipe_vector_db with args {'queries': ['asian fusion vegetarian recipe']}

[Model]: Based on the search results, here is a tasty Asian-inspired vegetarian recipe I can recommend:

Vegetarian Thai Curry

Ingredients:
- 1 tablespoon coconut oil
- 1 onion, diced 
- 3 cloves garlic, minced
- 1 tablespoon Thai red curry paste
- 1 (13.5 oz) can coconut milk
- 2 cups mixed vegetables (such as bell peppers, broccoli, carrots)
- 1 cup cooked chickpeas or tofu
- 1 tablespoon soy sauce or tamari
- 1 teaspoon brown sugar
- Juice of 1 lime
- Salt and pepper to taste
- Chopped cilantro for garnish

Instructions:
1. In a large skillet or wok, heat the coconut oil over medium heat. Add the onion and sauté for 2-3 minutes until translucent.
2. Add the garlic and Thai red curry paste. Cook for 1 minute, stirring constantly, until fragrant.
3. Pour in the coconut milk and

### Try with higher complexity / more restrictive prompts

In [86]:
test_query_5 = """
I am on a ketogenic diet and need a dinner recipe that is dairy-free, 
low in sodium, and takes less than an hour to cook.
"""

test_query_6 = """
I'm looking for a pescatarian main course that is low in saturated fat, 
uses Asian flavors, and can be prepared in under 45 minutes.
"""

test_query_7 = """
I need a diabetic-friendly, vegan breakfast recipe that is gluten-free, 
nut-free, and low in cholesterol, but also rich in omega-3 fatty acids 
and can be prepared the night before.
"""

test_query_8 = """
I am following a strict paleo diet and need a lunch recipe that is dairy-free, 
gluten-free, low in carbs, and low in sodium. Additionally, it should be rich in antioxidants, 
and can be made in under 30 minutes with minimal cooking equipment.
"""

In [87]:
test_query_response_5 = run_chat_loop(test_query_5)

[User]: 
I am on a ketogenic diet and need a dinner recipe that is dairy-free, 
low in sodium, and takes less than an hour to cook.

Model called query_food_recipe_vector_db with args {'queries': ['keto dairy-free low sodium dinner recipe']}

[Model]: Based on the search results, the recipe that best fits your criteria is the "Low Carb Chicken Salad". Here are the key details:

Ingredients:
- 2 cups cooked chicken, shredded or diced
- 1/4 cup mayonnaise (use a dairy-free version)
- 1 tbsp Dijon mustard
- 1 tbsp lemon juice
- 1/4 tsp salt
- 1/4 tsp black pepper
- 1/2 cup diced celery
- 1/4 cup diced onion
- 2 tbsp chopped parsley

Instructions:
1. In a medium bowl, mix together the cooked chicken, mayonnaise, Dijon mustard, lemon juice, salt, and pepper until well combined.
2. Stir in the diced celery, onion, and chopped parsley.
3. Serve chilled or at room temperature. Can be served on a bed of greens, in lettuce wraps, or with keto-friendly crackers.

This recipe is keto-friendly, dai

In [88]:
test_query_response_6 = run_chat_loop(test_query_6)

[User]: 
I'm looking for a pescatarian main course that is low in saturated fat, 
uses Asian flavors, and can be prepared in under 45 minutes.

Model called query_food_recipe_vector_db with args {'queries': ['pescatarian main course', 'low saturated fat', 'asian flavors', 'under 45 minutes']}

[Model]: Based on the search results, here is a recommended pescatarian main course recipe that meets your criteria:

Healthy Low Fat Baked Fish

Ingredients:
- 2 white fish fillets (such as tilapia or cod), about 1 lb total
- 1 tbsp olive oil
- 1 tbsp low-sodium soy sauce
- 1 tbsp rice vinegar
- 1 tsp sesame oil
- 1 tsp grated ginger
- 1 clove garlic, minced
- 1/4 tsp red pepper flakes (optional for spice)
- Salt and pepper to taste

Instructions:
1. Preheat oven to 400°F. Line a baking sheet with parchment paper.
2. In a small bowl, whisk together the olive oil, soy sauce, rice vinegar, sesame oil, ginger, garlic, and red pepper flakes (if using). 
3. Place the fish fillets on the prepared baki

In [89]:
test_query_response_7 = run_chat_loop(test_query_7)

[User]: 
I need a diabetic-friendly, vegan breakfast recipe that is gluten-free, 
nut-free, and low in cholesterol, but also rich in omega-3 fatty acids 
and can be prepared the night before.

Model called query_food_recipe_vector_db with args {'queries': ['diabetic-friendly vegan breakfast recipe', 'gluten-free nut-free low cholesterol breakfast', 'omega-3 rich breakfast recipe']}

[Model]: Based on the recipe metadata retrieved, here is a recommended diabetic-friendly, vegan, gluten-free, nut-free, low-cholesterol breakfast recipe that is rich in omega-3s and can be prepared the night before:

Overnight Oats with Chia Seeds and Berries

Ingredients:
- 1 cup rolled oats (gluten-free)
- 1 cup unsweetened almond milk or oat milk
- 2 tbsp chia seeds
- 1 tsp vanilla extract
- 1/2 tsp cinnamon
- 1 cup mixed berries (such as blueberries, raspberries, and blackberries)
- 1 tbsp maple syrup (optional)

Instructions:
1. In a medium bowl, combine the rolled oats, almond/oat milk, chia seeds, va

In [90]:
test_query_response_8 = run_chat_loop(test_query_8)

[User]: 
I am following a strict paleo diet and need a lunch recipe that is dairy-free, 
gluten-free, low in carbs, and low in sodium. Additionally, it should be rich in antioxidants, 
and can be made in under 30 minutes with minimal cooking equipment.

Model called query_food_recipe_vector_db with args {'queries': ['paleo lunch recipe', 'low carb paleo recipe', 'low sodium paleo recipe', 'antioxidant rich paleo recipe', '30 minute paleo recipe']}

[Model]: Based on the search results, here is a recommended paleo-friendly, low-carb, low-sodium lunch recipe that is rich in antioxidants and can be made in under 30 minutes:

Avocado and Chicken Paleo Pasta

Ingredients:
- 2 boneless, skinless chicken breasts
- 1 avocado, diced
- 1 cup cherry tomatoes, halved
- 1/2 cup fresh basil leaves, chopped
- 2 tbsp olive oil
- 1 tbsp lemon juice
- Salt and pepper to taste

Instructions:
1. Season the chicken breasts with salt and pepper.
2. Heat the olive oil in a skillet over medium-high heat. Add 

### Evaluation method
1. Use LLM as a judge 
2. Basic Sniff test

### Evaluation pipeline
1. Look at EDA result and determine the type of cuisine preesnt in the data. Come up with 8 "Test Questions". - Done. 
    test_query_1 to test_query_8
2. Feed the test questions in to the basic RAG, function calling RAG
3. Gather response and use the following two methods to evaluate them.
   1. LLM as a judge (provide grading ruberic) and ask the eval LLM to provide a score of 1-5.
   2. Sniff test 
  

### Recipe Grading Criteria - Pass into the Eval LLM as part of the prompt

Grading Scale (1-5)

5 - Exceptional Recipe:

    1. Accuracy: The recipe is highly accurate and closely matches the user's query, including all specified ingredients, dietary restrictions, and desired cuisine type.
    2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.
    3. Creativity: The recipe demonstrates creativity, offering a unique or interesting twist on a classic dish or a novel combination of ingredients.
    4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and any relevant tips or variations.
    5. Healthiness: The recipe provides a balanced nutritional profile, aligning with any specified health goals or dietary considerations.
    6. User Feedback: The recipe is likely to receive high ratings from users for both taste and ease of preparation.


4 - Very Good Recipe:

    1. Accuracy: The recipe mostly matches the user's query with minor deviations or substitutions that still align with the user's dietary restrictions and preferences.
    2. Clarity: The instructions are clear and easy to follow, with only minor areas that could benefit from additional detail.
    3. Creativity: The recipe shows some creativity and presents an appealing dish, though it may not be as unique as a 5-rated recipe.
    4. Completeness: The recipe includes most necessary details, but might miss a few minor tips or variations.
    5. Healthiness: The recipe is generally healthy, though it may not be as nutritionally balanced as a 5-rated recipe.
    6. User Feedback: The recipe is likely to receive good ratings from users, being tasty and reasonably easy to prepare.

3 - Good Recipe:

    1. Accuracy: The recipe has a reasonable match with the user's query but may include some inaccuracies or ingredient substitutions that slightly alter the dish's nature.
    2. Clarity: The instructions are generally clear but may have a few confusing steps or lack detailed guidance in some areas.
    3. Creativity: The recipe is standard with minimal creativity or uniqueness.
    4. Completeness: The recipe includes the essential details but lacks additional helpful information or suggestions.
    5. Healthiness: The recipe is moderately healthy but may lack balance in terms of nutritional profile.
    6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outstanding in taste or ease of preparation.

2 - Fair Recipe:

    1. Accuracy: The recipe has noticeable discrepancies from the user's query, potentially including ingredients that were supposed to be excluded due to dietary restrictions.
    2. Clarity: The instructions are unclear or difficult to follow, with significant gaps or ambiguities.
    3. Creativity: The recipe lacks creativity and may appear bland or uninspired.
    4. Completeness: The recipe is missing several important details, such as precise measurements or key preparation steps.
    5. Healthiness: The recipe is not particularly healthy and may have an unbalanced nutritional profile.
    6. User Feedback: The recipe is likely to receive below-average ratings due to issues with taste, clarity, or preparation difficulty.

1 - Poor Recipe:

    1. Accuracy: The recipe significantly deviates from the user's query, ignoring key dietary restrictions or preferences.
    2. Clarity: The instructions are confusing, incomplete, or incorrect, making the recipe difficult or impossible to follow.
    3. Creativity: The recipe is not creative and may seem haphazard or poorly thought out.
    4. Completeness: The recipe is missing critical details, such as major ingredients, steps, or cooking times.
    5. Healthiness: The recipe is unhealthy and lacks a balanced nutritional profile.
    6. User Feedback: The recipe is likely to receive low ratings due to poor taste, difficulty in preparation, or failure to meet user expectations.

In [92]:
# We will need to tune these prompts
# query_bedrock_llm() definition NEEDS TO BE RERUN
# each time when changes are made to this prompt

eval_message = f"""
You are a helpful assistant and expert in reviewing cooking recipes.

Please look at the given user query, recipe generated by another LLM and follow the ruberic below. Provide a score between 1-5 on how good the receipe is.

user query : "{test_query_1}"

Recipe generated by another LLM :"{test_query_response_1}"


Recipe review ruberic: 

Grading Scale (1-5)

5 - Exceptional Recipe:

1. Accuracy: The recipe is highly accurate and closely matches the user's query, including all specified ingredients, dietary restrictions, and desired cuisine type.
2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.
3. Creativity: The recipe demonstrates creativity, offering a unique or interesting twist on a classic dish or a novel combination of ingredients.
4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and any relevant tips or variations.
5. Healthiness: The recipe provides a balanced nutritional profile, aligning with any specified health goals or dietary considerations.
6. User Feedback: The recipe is likely to receive high ratings from users for both taste and ease of preparation.
4 - Very Good Recipe:

1. Accuracy: The recipe mostly matches the user's query with minor deviations or substitutions that still align with the user's dietary restrictions and preferences.
2. Clarity: The instructions are clear and easy to follow, with only minor areas that could benefit from additional detail.
3. Creativity: The recipe shows some creativity and presents an appealing dish, though it may not be as unique as a 5-rated recipe.
4. Completeness: The recipe includes most necessary details, but might miss a few minor tips or variations.
5. Healthiness: The recipe is generally healthy, though it may not be as nutritionally balanced as a 5-rated recipe.
6. User Feedback: The recipe is likely to receive good ratings from users, being tasty and reasonably easy to prepare.
3 - Good Recipe:

1. Accuracy: The recipe has a reasonable match with the user's query but may include some inaccuracies or ingredient substitutions that slightly alter the dish's nature.
2. Clarity: The instructions are generally clear but may have a few confusing steps or lack detailed guidance in some areas.
3. Creativity: The recipe is standard with minimal creativity or uniqueness.
4. Completeness: The recipe includes the essential details but lacks additional helpful information or suggestions.
5. Healthiness: The recipe is moderately healthy but may lack balance in terms of nutritional profile.
6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outstanding in taste or ease of preparation.
2 - Fair Recipe:

1. Accuracy: The recipe has noticeable discrepancies from the user's query, potentially including ingredients that were supposed to be excluded due to dietary restrictions.
2. Clarity: The instructions are unclear or difficult to follow, with significant gaps or ambiguities.
3. Creativity: The recipe lacks creativity and may appear bland or uninspired.
4. Completeness: The recipe is missing several important details, such as precise measurements or key preparation steps.
5. Healthiness: The recipe is not particularly healthy and may have an unbalanced nutritional profile.
6. User Feedback: The recipe is likely to receive below-average ratings due to issues with taste, clarity, or preparation difficulty.
1 - Poor Recipe:

1. Accuracy: The recipe significantly deviates from the user's query, ignoring key dietary restrictions or preferences.
2. Clarity: The instructions are confusing, incomplete, or incorrect, making the recipe difficult or impossible to follow.
3. Creativity: The recipe is not creative and may seem haphazard or poorly thought out.
4. Completeness: The recipe is missing critical details, such as major ingredients, steps, or cooking times.
5. Healthiness: The recipe is unhealthy and lacks a balanced nutritional profile.
6. User Feedback: The recipe is likely to receive low ratings due to poor taste, difficulty in preparation, or failure to meet user expectations.

"""

In [72]:
# Eval framework
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"

def query_bedrock_eval_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'messages': messages,
            'max_tokens': 3000,

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [75]:
qdrant_rag_chain_eval = (
    RunnableMap(
        # {"context": qdrant_retriever | format_docs,
         {"query": RunnablePassthrough()}
    )
    | process_prompt
    | query_bedrock_eval_llm
    # | parse_event_stream
)

In [77]:
qdrant_rag_chain_eval.invoke(eval_message)

{'id': 'msg_bdrk_01BhHqcAM6iQ6iCrxaHtCWLu',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-haiku-20240307',
 'content': [{'type': 'text',
   'text': 'Based on the recipe details provided and the review rubric, I would rate the "Low Carb Chicken Salad" recipe a 4 out of 5.\n\nRationale:\n\n1. Accuracy: The recipe closely matches the user\'s query, including all the specified dietary restrictions (dairy-free, gluten-free, low carb, low sodium) and desired attributes (rich in antioxidants, quick to prepare). There are no major deviations from the user\'s requirements.\n\n2. Clarity: The instructions provided are clear and easy to follow, with a logical sequence of steps. The cooking times and preparation details seem precise.\n\n3. Creativity: While the recipe is not overly creative, it presents a classic and satisfying paleo-friendly chicken salad that meets the user\'s needs. The combination of ingredients is straightforward but effective.\n\n4. Completeness: The recipe i

In [107]:
for i in range(1, 9):
    eval_message = f"""
    You are a helpful assistant and expert in reviewing cooking recipes.

    Please look at the given user query, recipe generated by another LLM and follow the rubric below. Provide a score between 1-5 on how good the recipe is.

    user query : "test_query_{i}"

    Recipe generated by another LLM : "test_query_response_{i}"


    Recipe review rubric: 

    Grading Scale (1-5)

    5 - Exceptional Recipe:

    1. Accuracy: The recipe is highly accurate and closely matches the user's query, including all specified ingredients, dietary restrictions, and desired cuisine type.
    2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.
    3. Creativity: The recipe demonstrates creativity, offering a unique or interesting twist on a classic dish or a novel combination of ingredients.
    4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and any relevant tips or variations.
    5. Healthiness: The recipe provides a balanced nutritional profile, aligning with any specified health goals or dietary considerations.
    6. User Feedback: The recipe is likely to receive high ratings from users for both taste and ease of preparation.
    4 - Very Good Recipe:

    1. Accuracy: The recipe mostly matches the user's query with minor deviations or substitutions that still align with the user's dietary restrictions and preferences.
    2. Clarity: The instructions are clear and easy to follow, with only minor areas that could benefit from additional detail.
    3. Creativity: The recipe shows some creativity and presents an appealing dish, though it may not be as unique as a 5-rated recipe.
    4. Completeness: The recipe includes most necessary details, but might miss a few minor tips or variations.
    5. Healthiness: The recipe is generally healthy, though it may not be as nutritionally balanced as a 5-rated recipe.
    6. User Feedback: The recipe is likely to receive good ratings from users, being tasty and reasonably easy to prepare.
    3 - Good Recipe:

    1. Accuracy: The recipe has a reasonable match with the user's query but may include some inaccuracies or ingredient substitutions that slightly alter the dish's nature.
    2. Clarity: The instructions are generally clear but may have a few confusing steps or lack detailed guidance in some areas.
    3. Creativity: The recipe is standard with minimal creativity or uniqueness.
    4. Completeness: The recipe includes the essential details but lacks additional helpful information or suggestions.
    5. Healthiness: The recipe is moderately healthy but may lack balance in terms of nutritional profile.
    6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outstanding in taste or ease of preparation.
    2 - Fair Recipe:

    1. Accuracy: The recipe has noticeable discrepancies from the user's query, potentially including ingredients that were supposed to be excluded due to dietary restrictions.
    2. Clarity: The instructions are unclear or difficult to follow, with significant gaps or ambiguities.
    3. Creativity: The recipe lacks creativity and may appear bland or uninspired.
    4. Completeness: The recipe is missing several important details, such as precise measurements or key preparation steps.
    5. Healthiness: The recipe is not particularly healthy and may have an unbalanced nutritional profile.
    6. User Feedback: The recipe is likely to receive below-average ratings due to issues with taste, clarity, or preparation difficulty.
    1 - Poor Recipe:

    1. Accuracy: The recipe significantly deviates from the user's query, ignoring key dietary restrictions or preferences.
    2. Clarity: The instructions are confusing, incomplete, or incorrect, making the recipe difficult or impossible to follow.
    3. Creativity: The recipe is not creative and may seem haphazard or poorly thought out.
    4. Completeness: The recipe is missing critical details, such as major ingredients, steps, or cooking times.
    5. Healthiness: The recipe is unhealthy and lacks a balanced nutritional profile.
    6. User Feedback: The recipe is likely to receive low ratings due to poor taste, difficulty in preparation, or failure to meet user expectations.
    """
    
    response = qdrant_rag_chain_eval.invoke(eval_message)
    print(response['content'])

    

[{'type': 'text', 'text': 'User query: "test_query_1"\n\nRecipe generated by another LLM: "test_query_response_1"\n\nRecipe Review:\n\nAfter carefully evaluating the recipe based on the provided rubric, I would give it a score of 3 - Good Recipe.\n\nRationale:\n\n1. Accuracy: The recipe has a reasonable match with the user\'s query, but there may be some minor inaccuracies or ingredient substitutions that slightly alter the dish\'s nature.\n\n2. Clarity: The instructions are generally clear, but there may be a few confusing steps or a lack of detailed guidance in some areas.\n\n3. Creativity: The recipe is standard with minimal creativity or uniqueness.\n\n4. Completeness: The recipe includes the essential details, but it lacks additional helpful information or suggestions.\n\n5. Healthiness: The recipe is moderately healthy, but it may lack balance in terms of nutritional profile.\n\n6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outsta